In [4]:
import os
import random

import numpy as np
import pandas as pd
from tqdm import tqdm

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, f1_score
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA

# Load data
data_path = r'E:\OneDrive\School\Technical Medicine\TM Jaar 1\Machine learning\TM10007_Machine_Learning_Group_4\TM\ecg_data.csv'
data = pd.read_csv(data_path, index_col=0)

# Splitting features and labels
x = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

outer = range(0, 200)
inner = range(0, 6)

best_params_list = {}

#prior split
X_train_main, X_test_main, y_train_main, y_test_main = train_test_split(
        x, y, test_size=0.2, stratify=y, shuffle=True, random_state=7)

# Apply PCA to reduce dimensionality
pca = PCA(n_components=0.99, random_state=7)  # Retain 95% of variance
X_train_main = pca.fit_transform(X_train_main)
X_test_main = pca.transform(X_test_main)

for outer_rand in tqdm(outer, desc='Outer Loop'):
    # Outer split
    X_train, X_test, y_train, y_test = train_test_split(
        X_train_main, y_train_main, test_size=0.2, stratify=y_train_main, shuffle=True, random_state=outer_rand)  # random_state added for reproducibility

    for inner_rand in tqdm(inner, desc='Inner Loop'):
        # Inner split
        X_train_train, X_train_val, y_train_train, y_train_val = train_test_split(
            X_train, y_train, test_size=0.15, stratify=y_train, shuffle=True, random_state=inner_rand)


        # APPLY SMOTE HERE (only to inner train set)
        smote = SMOTE(random_state=42)
        X_train_train_res, y_train_train_res = smote.fit_resample(X_train_train, y_train_train)

        # Hyperparameter grid
        param_grid_rf = {
            'n_estimators': [random.randint(50, 200) for _ in range(3)],
            'max_depth': [random.randint(5, 30) for _ in range(3)],
            'min_samples_split': [random.randint(2, 10) for _ in range(3)],
            'min_samples_leaf': [random.randint(1, 4) for _ in range(3)],
            'max_samples': [random.uniform(0.5, 1) for _ in range(3)],
            'bootstrap': [True]
        }

        rf_model = RandomForestClassifier()
        random_search = RandomizedSearchCV(
            estimator=rf_model,
            param_distributions=param_grid_rf,
            scoring='recall',
            cv=5,
            n_iter=10,
            n_jobs=-1
        )

        # Use SMOTE-resampled training data
        random_search.fit(X_train_train_res, y_train_train_res)

        best_params = random_search.best_params_
        best_params_list[f"Outer {outer_rand} - Inner {inner_rand}"] = best_params

        best_model = random_search.best_estimator_
        y_pred = best_model.predict(X_test)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        y_pred_proba = best_model.predict_proba(X_test)[:, 1]
        auc_score = roc_auc_score(y_test, y_pred_proba)
        f1 = f1_score(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)

        best_params_list[f"Outer {outer_rand} - Inner {inner_rand}"]['auc'] = auc_score
        best_params_list[f"Outer {outer_rand} - Inner {inner_rand}"]['f1'] = f1
        best_params_list[f"Outer {outer_rand} - Inner {inner_rand}"]['sensitivity'] = sensitivity
        best_params_list[f"Outer {outer_rand} - Inner {inner_rand}"]['specificity'] = specificity
        best_params_list[f"Outer {outer_rand} - Inner {inner_rand}"]['accuracy'] = accuracy

        print(f"\nOuter {outer_rand}, Inner {inner_rand} -> Best Hyperparameters: {best_params}")
        print(f"Test Set Accuracy: {accuracy:.4f}")

    # Save results
    results_df = pd.DataFrame.from_dict(best_params_list, orient='index')
    results_csv_path = os.path.join(os.getcwd(), 'best_hyperparameters_smote.csv')
    results_df.to_csv(results_csv_path)
    print(f"\nBest parameters saved to {results_csv_path} after Outer {outer_rand} completion")

# Summary
print("\nBest Parameters Summary:")
for key, val in best_params_list.items():
    print(f"{key}: {val}")



Outer Loop:   0%|          | 0/200 [00:00<?, ?it/s]


Outer 0, Inner 0 -> Best Hyperparameters: {'n_estimators': 67, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_samples': 0.940011580775287, 'max_depth': 11, 'bootstrap': True, 'auc': 0.8031345565749236, 'f1': 0.45, 'sensitivity': 0.375, 'specificity': 0.9357798165137615, 'accuracy': 0.8345864661654135}
Test Set Accuracy: 0.8346



Outer 0, Inner 1 -> Best Hyperparameters: {'n_estimators': 159, 'min_samples_split': 3, 'min_samples_leaf': 2, 'max_samples': 0.9021801963525973, 'max_depth': 16, 'bootstrap': True, 'auc': 0.8251146788990825, 'f1': 0.4761904761904762, 'sensitivity': 0.4166666666666667, 'specificity': 0.926605504587156, 'accuracy': 0.8345864661654135}
Test Set Accuracy: 0.8346



Outer 0, Inner 2 -> Best Hyperparameters: {'n_estimators': 63, 'min_samples_split': 8, 'min_samples_leaf': 1, 'max_samples': 0.6628486827975368, 'max_depth': 30, 'bootstrap': True, 'auc': 0.7893730886850152, 'f1': 0.48780487804878053, 'sensitivity': 0.4166666666666667, 'specificity': 0.9357798165137615, 'accuracy': 0.8421052631578947}
Test Set Accuracy: 0.8421



Outer 0, Inner 3 -> Best Hyperparameters: {'n_estimators': 199, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_samples': 0.945331914159374, 'max_depth': 24, 'bootstrap': True, 'auc': 0.8042813455657493, 'f1': 0.4615384615384615, 'sensitivity': 0.375, 'specificity': 0.944954128440367, 'accuracy': 0.8421052631578947}
Test Set Accuracy: 0.8421


...


Outer 7, Inner 5 -> Best Hyperparameters: {'n_estimators': 145, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_samples': 0.7321008382256177, 'max_depth': 12, 'bootstrap': True, 'auc': 0.8761467889908258, 'f1': 0.5128205128205129, 'sensitivity': 0.4166666666666667, 'specificity': 0.9541284403669725, 'accuracy': 0.8571428571428571}
Test Set Accuracy: 0.8571

Best parameters saved to e:\OneDrive\School\Technical Medicine\TM Jaar 1\Machine learning\TM10007_Machine_Learning_Group_4\TM\TM10007_Machine_Learning_Group_4-1\best_hyperparameters_smote.csv after Outer 7 completion



Outer 8, Inner 0 -> Best Hyperparameters: {'n_estimators': 184, 'min_samples_split': 7, 'min_samples_leaf': 3, 'max_samples': 0.5274555536217334, 'max_depth': 11, 'bootstrap': True, 'auc': 0.7801987767584099, 'f1': 0.5, 'sensitivity': 0.5, 'specificity': 0.8899082568807339, 'accuracy': 0.8195488721804511}
Test Set Accuracy: 0.8195



Outer 8, Inner 1 -> Best Hyperparameters: {'n_estimators': 114, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_samples': 0.9457300762932561, 'max_depth': 27, 'bootstrap': True, 'auc': 0.8012232415902141, 'f1': 0.3783783783783784, 'sensitivity': 0.2916666666666667, 'specificity': 0.944954128440367, 'accuracy': 0.8270676691729323}
Test Set Accuracy: 0.8271


Outer Loop:   4%|▍         | 8/200 [04:19<1:43:39, 32.39s/it]


KeyboardInterrupt: 

In [1]:
# %pip install scikit-learn==1.5.1
import sklearn
print(f"sklearn version: {sklearn.__version__}")

sklearn version: 1.5.1
